In [1]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sun Jun 27 22:12:02 2021

@author: Bojing Liu

Used first in Liu, B., Polack, M., et al. (2025).  Nature Communications, 16(1), 2328.
"""

import cv2
import os
import glob
import h5py
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.axes_grid1 import make_axes_locatable
import matplotlib.gridspec as gridspec
from sklearn.preprocessing import StandardScaler
import re
from sklearn.decomposition import PCA
from sklearn import metrics
from sklearn.metrics import pairwise_distances
from sklearn import datasets
# from sksurv.nonparametric import kaplan_meier_estimator
import numpy as np
import pandas as pd
import argparse
import sys
import gc

libs_path = '/gpfs/data/osmanlab/Processing/HPL/Histomorphological-Phenotype-Learning_TCGA'
sys.path.append(libs_path)
from data_manipulation.data import Data
#from models.visualization.clusters import plot_cluster_images, plot_wsi_clusters

In [8]:
def plot_cluster(h5_complete_path, by_event= True, pixel_size=0.252, value_cluster_ids_1=None, 
                 value_cluster_ids_0=None, 
                 meta_folder = 'cSCC102_v02a_tight', resolution=0.75, fold=1,
                 top_level='/gpfs/data/abl/deepomics/path_tto/1792px_xml_OUT_UCSF_B40_R90_40x',
                 tile_num = 8, cluster_plot_folder='visual_cluster',event_ind='outcome_binary', 
                 subplot_row_nr=1, dpi=300, figsize = None, filter_subset=None):
    
    # read .csv with cluster info
    main_cluster_path = h5_complete_path.split('hdf5_')[0]
    main_cluster_path = os.path.join(main_cluster_path, meta_folder)
    adatas_path       = os.path.join(main_cluster_path, 'adatas')
    
    groupby = 'leiden_%s' % resolution
    adata_name = h5_complete_path.split('/hdf5_')[1].split('.h5')[0] + '_%s__fold%s' % (groupby.replace('.', 'p'), fold) + '.csv'
    #print(adata_name)
    leiden_path = os.path.join(adatas_path, adata_name)
    frame_train = pd.read_csv(leiden_path)
    
    # get total number of clusters
    total_clusters = frame_train[groupby].max()+1
    print('total_clusters = ', total_clusters)
    
    # plots save in folder
    plot_root_folder =os.path.join(os.getcwd() ,cluster_plot_folder)
    if not os.path.exists(plot_root_folder):
        os.makedirs(plot_root_folder)
    else:
        pass
    
    if by_event== True:
        value_cluster_ids = dict()
        value_cluster_ids[0] = value_cluster_ids_0
        value_cluster_ids[1] = value_cluster_ids_1
        
    elif by_event== False:
        value_cluster_ids = dict() 
        value_cluster_ids[0] = list(range(total_clusters))
        
    # select cluster for each outcome types
    for i in value_cluster_ids.keys(): 
        for j in value_cluster_ids[i]:
            if j%2==0:
                print('cluster', j)            
            if by_event == True:
                df = frame_train[(frame_train[groupby].isin([j])) & (frame_train[event_ind]==i)]
            elif by_event == False: 
                df = frame_train[frame_train[groupby].isin([j])]

            # sample one tile from each patient if possible
            if filter_subset is not None:
                df = df[df.slides.str.contains(filter_subset)]
            try:
                if df.slides.nunique()>= tile_num:
                    df = df.drop_duplicates(subset=['slides']).reset_index(drop=True)
                    df = df.sample(frac=1).reset_index(drop=True)
                else:
                    df_sample = df.sample(frac=1).reset_index(drop=True)
            except:
                if df.combined_slides.nunique()>= tile_num:
                    df = df.drop_duplicates(subset=['combined_slides']).reset_index(drop=True)
                    df = df.sample(frac=3).reset_index(drop=True)
                else:
                    df_sample = df.sample(frac=1).reset_index(drop=True)
            #print(df)

            
            # plot 
            _df_ = df.iloc[0: tile_num].reset_index(drop=True)
            if figsize is not None:
                plt.figure(figsize = figsize)
            else:     
                plt.figure(figsize = (int(tile_num*4/subplot_row_nr), int(subplot_row_nr*4)))
            nIndx = 0
            print(_df_)
            for l in range(len(_df_)):
                try:
                    slide = _df_.slides.iloc[l]
                    tile = _df_.tiles.iloc[l]
                except:
                    slide = _df_.combined_slides.iloc[l]
                    tile = _df_.combined_tiles.iloc[l]
                slide=slide+"*_files"
                #print(slide)
                #print(tile)
                #img_path = os.path.join(top_level, slide, str(pixel_size), tile)
                #print(img_path)
                #print((os.path.join(top_level, "*/" + slide + "*", str(pixel_size), tile)))
                #print(glob.glob(os.path.join(top_level, slide , str(pixel_size), tile)))
                try:
                    img_path = glob.glob(os.path.join(top_level, slide , str(pixel_size), tile))[0]
                except:
                    print("glob failed: " + os.path.join(top_level, slide, str(pixel_size), tile))
                    continue
                try:
                    assert os.path.isfile(img_path)
                except:
                    print("img_path not found: " + img_path)
                    continue 
                x = cv2.imread(img_path)
                #x = Image.open(img_path)
                # ax = plt.subplot(subplot_row_nr, tile_num/subplot_row_nr, l+1)
                ax = plt.subplot(int(subplot_row_nr), int(tile_num/subplot_row_nr), nIndx+1)
                nIndx = nIndx +1
                plt.imshow(x)
                plt.axis('off')
            plt.subplots_adjust(wspace=0.05, hspace=0.05)
            plt.savefig(plot_root_folder + '/' + meta_folder +'_%s_fold%s_cluster%s' % (groupby.replace('.', 'p'), fold, j) + '.jpg', bbox_inches='tight', dpi=dpi)
            plt.close('all')
            print(plot_root_folder + '/' + meta_folder +'_%s_fold%s_cluster%s' % (groupby.replace('.', 'p'), fold, j) + '.jpg')
            gc.collect()
            
            


In [9]:
# path the h5 of the dataset projected on the trained network:
h5_complete_path = "/gpfs/data/osmanlab/Processing/HPL/Histomorphological-Phenotype-Learning_TCGA/results/BarlowTwins_3_twentyE/comb005_TCGA_40x_896px/h224_w224_n3_zdim128/hdf5_comb005_TCGA_40x_896px_he_complete_OS.h5"
# path to where jpeg tiles are saved
path_to_tiles_jpg = "/gpfs/scratch/coudrn01/NN_test/Melanoma_TCGA/Tiling/896px_0um2525_B40_20250221_TCGA"

plot_cluster(h5_complete_path = h5_complete_path,
             by_event= False,
             resolution=2.0,
             fold=3, 
             tile_num = 25,
             subplot_row_nr =  5,
             pixel_size = 0.2525,
             cluster_plot_folder='out_TCGA',
             filter_subset=None,
             top_level=path_to_tiles_jpg,
             meta_folder = 'comb005_v01')


total_clusters =  52
cluster 0
   data_source hist_subtype  indexes  labels original_set  os_event_data  \
0         TCGA         TCGA     1448     0.0        train       2.704997   
1         TCGA         TCGA     6466     0.0        valid       2.439425   
2         TCGA         TCGA     1123     0.0         test       9.620808   
3         TCGA         TCGA      584     0.0        valid       1.856263   
4         TCGA         TCGA     8306     0.0        train       1.314168   
5         TCGA         TCGA      237     0.0         test       5.264887   
6         TCGA         TCGA      532     0.0        train       0.982888   
7         TCGA         TCGA      807     0.0         test       0.648871   
8         TCGA         TCGA       10     0.0        train       7.972621   
9         TCGA         TCGA      150     0.0        train       1.333333   
10        TCGA         TCGA     6799     0.0        valid       7.238877   
11        TCGA         TCGA    44314     0.0        valid

/gpfs/data/osmanlab/Processing/HPL/Histomorphological-Phenotype-Learning_TCGA/utilities/visualizations/out_TCGA/comb005_v01_leiden_2p0_fold3_cluster1.jpg
cluster 2
   data_source hist_subtype  indexes  labels original_set  os_event_data  \
0         TCGA         TCGA      614     0.0        train       1.308693   
1         TCGA         TCGA    14735     0.0         test       0.648871   
2         TCGA         TCGA    46741     0.0        train       3.277207   
3         TCGA         TCGA      130     0.0        train       9.374401   
4         TCGA         TCGA      611     0.0        valid       0.988364   
5         TCGA         TCGA      290     0.0         test       0.648871   
6         TCGA         TCGA      410     0.0        valid       8.626968   
7         TCGA         TCGA     6173     0.0        valid       5.979466   
8         TCGA         TCGA     2625     0.0        train       2.704997   
9         TCGA         TCGA      865     0.0        valid       0.777550   


/gpfs/data/osmanlab/Processing/HPL/Histomorphological-Phenotype-Learning_TCGA/utilities/visualizations/out_TCGA/comb005_v01_leiden_2p0_fold3_cluster3.jpg
cluster 4
   data_source hist_subtype  indexes  labels original_set  os_event_data  \
0         TCGA         TCGA    12460     0.0        train       4.429843   
1         TCGA         TCGA    87586     0.0         test       2.338125   
2         TCGA         TCGA      841     0.0        train       5.503080   
3         TCGA         TCGA    23909     0.0        train      11.304586   
4         TCGA         TCGA    11540     0.0        train       1.694730   
5         TCGA         TCGA      625     0.0         test       4.457221   
6         TCGA         TCGA     1727     0.0        train       5.535934   
7         TCGA         TCGA    21477     0.0        valid       2.042437   
8         TCGA         TCGA    10073     0.0        valid       1.535934   
9         TCGA         TCGA     5547     0.0         test      13.407255   


/gpfs/data/osmanlab/Processing/HPL/Histomorphological-Phenotype-Learning_TCGA/utilities/visualizations/out_TCGA/comb005_v01_leiden_2p0_fold3_cluster5.jpg
cluster 6
   data_source hist_subtype  indexes  labels original_set  os_event_data  \
0         TCGA         TCGA      937     0.0         test       8.695414   
1         TCGA         TCGA     3059     0.0        valid       3.364819   
2         TCGA         TCGA    21659     0.0        valid       0.303901   
3         TCGA         TCGA    33953     0.0        valid       4.120465   
4         TCGA         TCGA     1336     0.0        train       8.922656   
5         TCGA         TCGA     1283     0.0         test       0.867899   
6         TCGA         TCGA    23875     0.0         test       0.750171   
7         TCGA         TCGA    13894     0.0        train       1.979466   
8         TCGA         TCGA    54262     0.0        train       9.374401   
9         TCGA         TCGA     1595     0.0        valid       4.120465   


/gpfs/data/osmanlab/Processing/HPL/Histomorphological-Phenotype-Learning_TCGA/utilities/visualizations/out_TCGA/comb005_v01_leiden_2p0_fold3_cluster7.jpg
cluster 8
   data_source hist_subtype  indexes  labels original_set  os_event_data  \
0         TCGA         TCGA    32664     0.0        valid       0.221766   
1         TCGA         TCGA      277     0.0        train       1.204654   
2         TCGA         TCGA     3355     0.0        train       5.453799   
3         TCGA         TCGA      882     0.0        valid       3.364819   
4         TCGA         TCGA     2352     0.0         test       1.494867   
5         TCGA         TCGA     2369     0.0        train       5.229295   
6         TCGA         TCGA    11407     0.0        train       4.084873   
7         TCGA         TCGA     1267     0.0        train       2.568104   
8         TCGA         TCGA     6639     0.0        train       5.535934   
9         TCGA         TCGA      191     0.0        valid       8.432580   


...

/gpfs/data/osmanlab/Processing/HPL/Histomorphological-Phenotype-Learning_TCGA/utilities/visualizations/out_TCGA/comb005_v01_leiden_2p0_fold3_cluster43.jpg
cluster 44
   data_source hist_subtype  indexes  labels original_set  os_event_data  \
0         TCGA         TCGA    18278     0.0        valid       8.432580   
1         TCGA         TCGA     2938     0.0         test       1.508556   
2         TCGA         TCGA     5418     0.0        train       5.169062   
3         TCGA         TCGA    20058     0.0        train       8.922656   
4         TCGA         TCGA      520     0.0         test       9.620808   
5         TCGA         TCGA   144959     0.0        train       2.001369   
6         TCGA         TCGA   132983     0.0        valid       8.626968   
7         TCGA         TCGA    28471     0.0        valid       4.071184   
8         TCGA         TCGA      634     0.0         test       3.504449   
9         TCGA         TCGA   140112     0.0        train       0.982888  

/gpfs/data/osmanlab/Processing/HPL/Histomorphological-Phenotype-Learning_TCGA/utilities/visualizations/out_TCGA/comb005_v01_leiden_2p0_fold3_cluster45.jpg
cluster 46
   data_source hist_subtype  indexes  labels original_set  os_event_data  \
0         TCGA         TCGA    21466     0.0         test       4.947296   
1         TCGA         TCGA      933     0.0        train       1.782341   
2         TCGA         TCGA      910     0.0        train       2.704997   
3         TCGA         TCGA      583     0.0        valid       4.123203   
4         TCGA         TCGA     8008     0.0         test       5.872690   
5         TCGA         TCGA    20187     0.0         test       0.843258   
6         TCGA         TCGA    31735     0.0        valid       1.007529   
7         TCGA         TCGA    80912     0.0        train       9.330595   
8         TCGA         TCGA     1491     0.0        train      13.965777   
9         TCGA         TCGA   125886     0.0         test      10.795346  

/gpfs/data/osmanlab/Processing/HPL/Histomorphological-Phenotype-Learning_TCGA/utilities/visualizations/out_TCGA/comb005_v01_leiden_2p0_fold3_cluster47.jpg
cluster 48
   data_source hist_subtype  indexes  labels original_set  os_event_data  \
0         TCGA         TCGA    88899     0.0        valid       0.585900   
1         TCGA         TCGA     7778     0.0        train      11.485284   
2         TCGA         TCGA     1200     0.0        train       1.883641   
3         TCGA         TCGA     3643     0.0        train      13.867214   
4         TCGA         TCGA     3582     0.0        valid      10.425736   
5         TCGA         TCGA     9171     0.0        valid       3.649555   
6         TCGA         TCGA    38206     0.0        valid       3.364819   
7         TCGA         TCGA    57431     0.0         test       5.264887   
8         TCGA         TCGA     4984     0.0        train       8.438056   
9         TCGA         TCGA     1027     0.0        train      14.559890  

/gpfs/data/osmanlab/Processing/HPL/Histomorphological-Phenotype-Learning_TCGA/utilities/visualizations/out_TCGA/comb005_v01_leiden_2p0_fold3_cluster49.jpg
cluster 50
   data_source hist_subtype  indexes  labels original_set  os_event_data  \
0         TCGA         TCGA     2521     0.0        valid       4.120465   
1         TCGA         TCGA   145522     0.0        train       0.591376   
2         TCGA         TCGA     3224     0.0        train       2.266940   
3         TCGA         TCGA    18350     0.0        valid       0.777550   
4         TCGA         TCGA       35     0.0         test       1.779603   
5         TCGA         TCGA    40639     0.0         test       0.648871   
6         TCGA         TCGA      108     0.0        train       7.570157   
7         TCGA         TCGA    18145     0.0        valid       4.071184   
8         TCGA         TCGA    24258     0.0        train      14.559890   
9         TCGA         TCGA    11326     0.0        train       2.239562  

/gpfs/data/osmanlab/Processing/HPL/Histomorphological-Phenotype-Learning_TCGA/utilities/visualizations/out_TCGA/comb005_v01_leiden_2p0_fold3_cluster51.jpg
